In [2]:
from sympy.abc import x,y,z
from sympy import lambdify,diff, sqrt, simplify
import numpy as np
import plotly.graph_objects as go
from sklearn.cluster import DBSCAN
from scipy.optimize import minimize
import trimesh as tm
from skimage import measure

def compute_jet(ff):
    f = lambdify([x,y,z],ff)
    fx = lambdify([x,y,z], diff(ff,x))
    fy = lambdify([x,y,z], diff(ff,y))
    fz = lambdify([x,y,z], diff(ff,z))
    return f, fx, fy, fz

def compute_jet_numerical(ff, xs, ys, zs):
    f, fx, fy, fz = compute_jet(ff)
    fxs = fx(xs,ys,zs)
    fys = fy(xs,ys,zs)
    fzs = fz(xs,ys,zs)
    ns = np.sqrt(fxs**2 + fys**2 + fzs**2)
    return f(xs,ys,zs), fxs/ns, fys/ns, fzs/ns

def grid_xyz(box, res):
    xmin, xmax, ymin, ymax, zmin, zmax = box
    xs = np.linspace(xmin, xmax, res)
    ys = np.linspace(ymin, ymax, res)
    zs = np.linspace(zmin, zmax, res)
    X,Y,Z = np.meshgrid(xs, ys, zs)
    X = X.flatten()
    Y = Y.flatten()
    Z = Z.flatten()
    return X, Y, Z

def iso_surface(f, box, points_count, res = 150):
    x_min, x_max, y_min, y_max, z_min, z_max = box
    x = np.linspace(x_min, x_max, res)
    y = np.linspace(y_min, y_max, res)
    z = np.linspace(z_min, z_max, res)
    X, Y, Z = np.meshgrid(x, y, z)
    vertices, faces, _, _ = measure.marching_cubes(f(X,Y,Z), level=0, spacing=(1.0, 1.0, 1.0))
    vertices[:,0] = vertices[:,0] * (box[1] - box[0]) / res + box[0]
    vertices[:,1] = vertices[:,1] * (box[3] - box[2]) / res + box[2]
    vertices[:,2] = vertices[:,2] * (box[5] - box[4]) / res + box[4]
    mesh = tm.Trimesh(vertices, faces)
    vs, fs = tm.sample.sample_surface_even(mesh, points_count)
    return vs, fs

def compute_align_energy(normal, f):
    f, fx, fy, fz = compute_jet(f)
    def align_energy(point):
        xv = point[0]
        yv = point[1]
        zv = point[2]
        fv = f(xv, yv, zv)
        fxv = fx(xv, yv, zv)
        fyv = fy(xv, yv, zv)
        fzv = fz(xv, yv, zv)
        cp = np.cross([fxv, fyv, fzv], normal)
        return np.sqrt(cp[0]**2 + cp[1]**2 + cp[2]**2) + fv**2
    return align_energy


def compute_inverse_point(formula,normal,points0):
    f, fx,fy,fz = compute_jet(formula)
    x0s = points0[:,0]
    y0s = points0[:,1]
    z0s = points0[:,2]
    nxs = fx(x0s,y0s,z0s)
    nys = fy(x0s,y0s,z0s)
    nzs = fz(x0s,y0s,z0s)
    ns = (nxs**2 + nys**2 + nzs**2)**0.5
    nxs = nxs/ns
    nys = nys/ns
    nzs = nzs/ns
    sol = np.argmin((nxs-normal[0])**2 + (nys-normal[1])**2 + (nzs-normal[2])**2)
    align_energy = compute_align_energy(normal, formula)
    res = minimize(align_energy, (x0s[sol], y0s[sol], z0s[sol]))
    return res.x

def get_spherical_triangle_condition(n0,n1,n2):
    n01 = np.cross(n0,n1)
    n12 = np.cross(n1,n2)
    n20 = np.cross(n2,n0)
    def cond(x,y,z):
        c01 = x*n01[0] + y*n01[1] + z*n01[2] > 0
        c12 = x*n12[0] + y*n12[1] + z*n12[2] > 0
        c20 = x*n20[0] + y*n20[1] + z*n20[2] > 0
        return c01 & c12 & c20
    return cond

def gauss_inv_region(formula, condition, points):
    f, fx, fy, fz = compute_jet(formula)
    xs = points[:,0]
    ys = points[:,1]
    zs = points[:,2]
    nxs = fx(xs,ys,zs)
    nys = fy(xs,ys,zs)
    nzs = fz(xs,ys,zs)
    ns = np.sqrt(nxs**2 + nys**2 + nzs**2)
    nxs = nxs/ns
    nys = nys/ns
    nzs = nzs/ns
    sol, = np.where(condition(nxs, nys, nzs))
    return sol



def cluster_points(sph_vs, points, min_dst):
    points_clusters = []
    for n in sph_vs:
        nx = n[0]
        ny = n[1]
        nz = n[2]
        nrm = (nx**2 + ny**2 + nz**2)**0.5
        nx = nx/nrm
        ny = ny/nrm
        nz = nz/nrm
        dsts = np.sqrt((points[:,0] - nx)**2 + (points[:,1] - ny)**2 + (points[:,2] - nz)**2)
        ids, = np.where(dsts < min_dst)
        points_clusters.append(ids)
    return points_clusters

# Sphere triangulation

In [3]:

res = 2
# Load the data
base = "../SphereTriangulation/"
sph_fs = np.loadtxt(base+'triangles%s.txt'%res).astype(int)
sph_vs = np.loadtxt(base+'vertices%s.txt'%res)

print("Spherical points count: %s"%sph_vs.shape[0])

sp_mesh = tm.Trimesh(sph_vs, sph_fs)
vnghs = sp_mesh.vertex_neighbors
d_min = 100000
for i in range(len(sph_vs)):
    nghs = np.array(vnghs[i])
    n = sph_vs[i]
    d = np.min(np.linalg.norm(n-sph_vs[nghs]))
    if d < d_min:
        d_min = d

print("Minimal distance between points: %s"%d_min)

# plot shperical triangles
fig = go.Figure()
fig.add_trace(go.Mesh3d(x=sph_vs[:,0], y=sph_vs[:,1], z=sph_vs[:,2], i=sph_fs[:,0], j=sph_fs[:,1], k=sph_fs[:,2], color='gray', opacity=1.0, flatshading=True))
fig.add_trace(go.Scatter3d(x=sph_vs[:,0], y=sph_vs[:,1], z=sph_vs[:,2], mode='markers', marker=dict(size=2, color='red')))
fig.update_layout(width=600, height=600)
fig.show()

Spherical points count: 162
Minimal distance between points: 0.6169411820917984


# Isosurface

In [ ]:
R = 2
r = 1
ax = .1
ay = 0
formula = (sqrt(x**2+y**2)- R)**2 + (z-ax*x**2+ay*y**2)**2 - r**2
#formula = (x/2)**2+y**2 + z**2 - r**2
box = [-3,3,-3,3,-1.5,1.5]
f, fx, fy, fz = compute_jet(formula)

srf_vs, faces = iso_surface(f, box, 100000)

fig = go.Figure()
srf_grph = go.Scatter3d(x=srf_vs[:,0], y=srf_vs[:,1], z=srf_vs[:,2],marker=dict(size=1, color='gray'), mode='markers', name='Surface')
fig.add_trace(srf_grph)
fig.update_layout(width=800, height=800)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(srf_grph)


for i in range(len(sph_fs)):
    n0,n1,n2 = sph_vs[sph_fs[i]]
    cond = get_spherical_triangle_condition(n0,n1,n2)
    inv_region = gauss_inv_region(formula, cond, srf_vs)
    ss = srf_vs[inv_region]
    fig.add_trace(go.Scatter3d(x=ss[:,0], y=ss[:,1], z=ss[:,2], mode='markers', marker=dict(size=2)))

fig.update_layout(width=600, height=600)
fig.show()

# Clusterning points

In [ ]:
nxs = fx(srf_vs[:,0], srf_vs[:,1], srf_vs[:,2])
nys = fy(srf_vs[:,0], srf_vs[:,1], srf_vs[:,2])
nzs = fz(srf_vs[:,0], srf_vs[:,1], srf_vs[:,2])
norms = np.sqrt(nxs**2 + nys**2 + nzs**2)
nxs = nxs/norms
nys = nys/norms
nzs = nzs/norms
srf_ns = np.array([nxs,nys,nzs]).T

clusters = cluster_points([[0,1,0],[0,0,1]], srf_ns,.025)


fig = go.Figure()
fig.add_trace(srf_grph)
for c in clusters:
    fig.add_trace(go.Scatter3d(x=srf_vs[c,0], y=srf_vs[c,1], z=srf_vs[c,2], mode='markers', marker=dict(size=2.0), name='Clusters'))


fig.update_layout(width=800, height=800)
fig.show()

In [ ]:
np.linalg.norm(srf_ns[30])

In [ ]:
sols = []
for n in sph_vs:
    p = compute_inverse_point(formula, n, vertices)
    sols.append(p)

In [ ]:
sols = np.array(sols)
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=sols[:,0], y=sols[:,1], z=sols[:,2], mode='markers', marker=dict(size=2, color='red')))
fig.update_layout(width=800, height=800)
fig.show()

# Surface

In [ ]:
X, Y, Z = grid_xyz(box,100)
f, nx, ny, nz = compute_jet_numerical(formula, X, Y, Z)

cf = np.abs(f) < 0.05
surface_trace = go.Scatter3d(x=X[cf], y=Y[cf], z=Z[cf], mode='markers', marker=dict(size=1, color='orange'))

print("Surface points count", np.sum(cf))
fig = go.Figure()
fig.add_trace(surface_trace)
fig.update_layout(width=800, height=800)
fig.show()

# Inverse Gauss map for points

In [ ]:
ns = sph_vs
inverse_points_aprox = []
size_max = 0
size_min = np.infty
for p in ns:
    cn = np.sqrt((nx-p[0])**2 + (ny-p[1])**2 + (nz-p[2])**2) < 0.05
    cfn = cf & cn
    sz =  np.sum(cfn)
    if sz < size_min:
        size_min = sz
    if sz > size_max:
        size_max = sz
    
    xv = X[cfn]
    yv = Y[cfn]
    zv = Z[cfn]
    inverse_points_aprox.append(np.array([xv,yv,zv]).T)

print("Size min:", size_min)
print("Size max:", size_max)

In [ ]:
def align_inverse_points(eq,normal, inv_points_aproxs):
    x_inv = inv_points_aproxs[:,0]
    y_inv = inv_points_aproxs[:,1]
    z_inv = inv_points_aproxs[:,2]
    align_energy = compute_align_energy(normal, eq)
    inv_points = []
    cnt = len(x_inv)
    for i in range(cnt):
        res = minimize(align_energy, (x_inv[i], y_inv[i], z_inv[i]))
        inv_points.append(res.x)

    inv_points = np.array(inv_points)
    clusters = DBSCAN(eps=0.1, min_samples=2).fit(inv_points)
    labels = np.unique(clusters.labels_)
    inv_points_clustered = []
    for l in labels:
        if l == -1:
            continue
        cluster = inv_points[clusters.labels_ == l]
        inv_points_clustered.append(cluster.mean(axis=0))

    return np.array(inv_points_clustered)

all_inv_points = []
for i in range(len(sph_vs)):   
    inv_points = align_inverse_points(formula, sph_vs[i], inverse_points_aprox[i])
    all_inv_points.extend(inv_points)

inv_points = np.array(all_inv_points)

In [ ]:
fig = go.Figure()
#fig.add_trace(surface_trace)
fig.add_trace(go.Scatter3d(x=inv_points[:,0], y=inv_points[:,1], z=inv_points[:,2], mode='markers', marker=dict(size=2, color='red')))
fig.update_layout(width=800, height=800)

In [ ]:
vertices, faces = iso_surface(f, box, 1000)
faces